In [ ]:
# This module forward C++ errors to Python to help debugging
%load_ext wurlitzer

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import jiminy_py.core as jiminy  # The main module of jiminy - this is what gives access to the Robot
from jiminy_py.simulator import Simulator

from gym_jiminy.toolbox.math import quat_to_yaw

# Flexible arm

## Instantiate a robot

In [ ]:
# First mount the drive
urdf_path = 'flexible_arm.urdf'
robot = jiminy.Robot()
robot.initialize(urdf_path, has_freeflyer=False)

# Add motor
motor_joint_name = 'base_to_link1'
motor = jiminy.SimpleMotor(motor_joint_name)
robot.attach_motor(motor)
motor.initialize(motor_joint_name)

# Add sensor
encoder = jiminy.EncoderSensor('active_joint')
robot.attach_sensor(encoder)
encoder.initialize('base_to_link1')

In [ ]:
# We set inertia along non-moving axis to 1.0 for numerical stability 
k_j = 100.0
d_j = 5.0
model_options = robot.get_model_options()
model_options['dynamics']['flexibilityConfig'] = [{
    'frameName': f"link{i}_to_link{i+1}",
    'stiffness': k_j * np.ones(3),
    'damping': d_j * np.ones(3),
    'inertia': np.array([1.0, 1.0, 0.0])
} for i in range(1,5)]
model_options['joints']['enablePositionLimit'] = False
model_options['joints']['enableVelocityLimit'] = False
robot.set_model_options(model_options)

## Instantiate a controller

In [ ]:
# Define the command: for now, the motor is off and doesn't modify the output torque.
Kp = 150.
Kd = 5.
q_ref = np.pi/4
def compute_command(t, q, v, sensors_data, command):
    q_ref = np.pi/4
    q_a = sensors_data['EncoderSensor'][0]
    v_a = sensors_data['EncoderSensor'][1]
    command[:] = Kp*(q_ref - q_a) - Kd*v_a

In [ ]:
# Instantiate and initialize the controller
controller = jiminy.ControllerFunctor(compute_command)
controller.initialize(robot)

jiminy_py.core.hresult_t.SUCCESS

## Instantiate a simulator

In [ ]:
# Create a simulator using this robot and controller
simulator = Simulator(robot, controller)

In [ ]:
# Configure the integrator
engine_options = simulator.engine.get_options()
engine_options['stepper']['controllerUpdatePeriod'] = 0.001
engine_options['stepper']['odeSolver'] = 'runge_kutta_4'
engine_options['stepper']['dtMax'] = 0.000125
simulator.engine.set_options(engine_options)

In [ ]:
# print out all the available engine options
engine_options

## Simulate the system

In [ ]:
# Set initial condition and simulation length
q0, v0 = np.array([0.]), np.array([0.])
duration = 2.0

# Launch the simulation
simulator.simulate(duration, q0, v0, is_state_theoretical=True)

In [ ]:
# Replay the result
camera_xyzrpy = ([0.0, -1.5, 2.0e-5], [np.pi/2, 0.0, 0.0])
simulator.replay(camera_xyzrpy=camera_xyzrpy, speed_ratio=0.5)

In [ ]:
# Get dictionary of logged scalar variables
log_data = simulator.log_data

# Display the true position and velocity of the actuated joint
_, (ax1, ax2) = plt.subplots(1,2, figsize=(12,4))
ax1.plot(log_data['Global.Time'], log_data['HighLevelController.currentPositionbase_to_link1'])
ax1.grid()
ax2.plot(log_data['Global.Time'], log_data['HighLevelController.currentVelocitybase_to_link1'])
ax2.grid()
plt.show()

# Display the true effort at the actuated joint. 
# It includes the internal dynamics if any (friction, plus custom if any).
_, ax = plt.subplots()
ax.plot(log_data['Global.Time'], log_data['HighLevelController.currentEffortbase_to_link1'])
ax.grid()
ax.set_title('Torque (Nm)')
plt.show()

In [ ]:
# Extract the deformation angles along rotation axis
qf = []
for fj_name, fj_idx in zip(robot.flexible_joints_names, robot.flexible_joints_idx):
    quat = (log_data[f'HighLevelController.currentPosition{fj_name}QuatX'],
            log_data[f'HighLevelController.currentPosition{fj_name}QuatY'],
            log_data[f'HighLevelController.currentPosition{fj_name}QuatZ'],
            log_data[f'HighLevelController.currentPosition{fj_name}QuatW'])
    idx_q = robot.pinocchio_model.joints[fj_idx].idx_q
    cos_yaw = - 1.0 + 2.0 * (quat[3] ** 2 + quat[0] ** 2)
    sin_yaw = 2.0 * (quat[2] * quat[3] + quat[0] * quat[1])
    qf.append(np.arctan2(sin_yaw, cos_yaw))
qf = np.stack(qf, axis=0)

In [ ]:
# Display the deformation angles
for k, column in enumerate(qf.T):
   plt.plot(log_data['Global.Time'], column, label=f'q{k}')
plt.grid()
plt.legend()
plt.title('Flexible joints angles (rad)')
plt.show()